In [ ]:
##Importando bibliotecas
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib
from datetime import date
matplotlib.rcParams['figure.figsize'] = (18,8)

# Import do arquivo e ajustes na base

In [ ]:
#Fazendo o upload do arquivo trades.xlsx
arquivo = pd.read_excel(r'carteira.xlsx', sheet_name='VISTA')
arquivo.columns = arquivo.iloc[0]
arquivo = arquivo.iloc[2:] 
arquivo.index = arquivo["Data"]
arquivo.columns = ["Data", "ticker", "C/V", "Quantidade", "Unitário (s corretagem)", 
                    "Total (s corretagem)", "Unitário (c corretagem)", "Total (c corretagem)", "Total Despesas", "NaN"]
arquivo.drop(columns=["Data", "Unitário (s corretagem)", "Total (s corretagem)", "Total Despesas", "NaN"], inplace=True)

# Operações

In [ ]:
### Criando tabela com colunas para cada ativo e indexando por data
trade_quantidade = pd.pivot_table(arquivo, values='Quantidade', index='Data', columns='ticker', aggfunc=np.sum, fill_value=0)
trade_quantidade

In [ ]:
### Criando tabela com os preços de compra e venda
trade_preco = pd.pivot_table(arquivo, values="Unitário (c corretagem)", index=['Data'], columns=arquivo['ticker'].str.upper(), aggfunc=np.sum,fill_value=0)
trade_preco

## Download dos dados diários

In [ ]:
### Baixando os cotações das ações
prices = yf.download(tickers=(trade_quantidade.columns+'.SA').to_list(), start=trade_quantidade.index[0], rounding=True)['Adj Close']

## Consolidadndo posições

In [ ]:
### Consolida posições
prices.columns  = prices.columns.str.rstrip('.SA')
prices.dropna(how='all', inplace=True)
trades = trade_quantidade.reindex(index=prices.index)
trades.fillna(value=0, inplace=True)
aportes = (trades * trade_preco).sum(axis=1)
posicao = trades.cumsum()

In [ ]:
### Consolida saldo
carteira = posicao * prices
carteira['saldo'] = carteira.sum(axis=1)
#carteira = carteira[:-2]

### Exporta para Excel
data_atual = date.today()
carteira.to_excel('carteira_{}-{}-{}.xlsx'.format(data_atual.day, data_atual.month,data_atual.year))

# Volatilidade

## Retorno e correlação

In [ ]:
#Baixa os dados históricos
hist_price = yf.download(tickers=(trade_quantidade.columns+'.SA').to_list(), period="3y")['Adj Close']
retr_log = np.log(hist_price).diff()
retr_log.dropna(inplace=True)

#Calcula o peso de cada ação  - dados históricos de movimentações
weights = carteira.div(carteira['saldo'], axis=0)
weights = weights.iloc[:,:-1]
weights = weights.drop(weights.index[278])
weights.columns = retr_log.columns

#Calcula a volatilidade EWMA para todos os ativos em uma janela de 252 dias
r = retr_log.ewm(alpha = 0.05, min_periods=252, adjust=False).std(252)
vol  = (r)**(1/1)
vol['2020':].plot(figsize=(30,10))

#Baixa os dados e calcula volatilidade para o IBOVESPA
prices = yf.download(tickers= "^BVSP", period="3y", rounding=True)['Adj Close']
retr_log_ibov = np.log(prices).diff()
retr_log_ibov.dropna(inplace=True)
ibov_vol = retr_log_ibov.ewm(alpha = 0.05, min_periods=252, adjust=False).std(252)
ibov_vol  = (ibov_vol)**(1/1)

#Calculo de correlação entre os ativosb
corr = retr_log.rolling(252).corr()
new_index = pd.MultiIndex.from_tuples(corr.index, names=['Date','Ticker'])
corr_ewma = pd.DataFrame(corr, columns = corr.columns, index=new_index)

#limpa dados NA
vol.dropna( inplace=True)
vol = vol[weights.index[0]:weights.index[-1]]

In [ ]:
#Calcula a volatilidade da carteira do Sr. Carlos utilizando matrizes
a=-1
n = 0
vol_cart = pd.DataFrame(columns=list(weights.columns.values), index=range(len(weights.index)))

In [ ]:
len(vol.index)

In [ ]:
corr

## Calcula vol da carteira

In [ ]:
#Calcula o corr*pesos par a par ( ex: BBDC4 x BRFS4 .... PETR4 x VALE3)
for n in range(len(vol)):
    vol_cart.loc[n] = (vol.iloc[n,]*weights.iloc[n,])*corr.loc[weights.index[n,]].dot(weights.iloc[n,])
    n = n+1
    if n == (len(vol)): break

In [ ]:
vol_cart = pd.DataFrame(vol_cart.sum(axis=1))
vol_cart.index = vol.index

In [ ]:
#Adiciona o IBOV para Benchmark
vol_cart['IBOV'] = ibov_vol[weights.index[0]:weights.index[-1]]

In [ ]:
vol_cart.plot(figsize = (20,10))

In [ ]:
vol_cart.to_excel('vol_carteira_{}-{}-{}.xlsx'.format(data_atual.day, data_atual.month,data_atual.year))